In [1]:
from wayne_utils import load_data, save_data
from tqdm import tqdm
import os
import re
'''os.chdir( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL")
from exec_post.nebula_execute import DB_execute'''

tests = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/test_data_shuffle.jsonl", "jsonl")
trains = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/data/N3/train_data_shuffle.jsonl", "jsonl")
datasets_schema = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/edge_vertex.json", "json")

test_gql_list = [ tests[i]["content"] for i in range(len(tests))]       # 1254  # 去重后511
train_gql_list = [ trains[i]["content"] for i in range(len(trains))]    # 3862  # 822

disease_gql_list = []           # 221 / 286
nba_gql_list = []               # 597 / 3670
potter_gql_list = []            # 231 / 236
others_gql_list = []            # 252 / 924
for lists in [ tests, trains]:
    for i in range(len(lists)):
        schema = lists[i]['schema']
        if "player(Tag)" in schema:
            nba_gql_list.append( lists[i]["content"] )
        elif "class drug" in schema:
            disease_gql_list.append( lists[i]["content"] )
        elif "class character" in schema:
            potter_gql_list.append( lists[i]["content"] )
        else:
            # raise Exception( f"{i}不合法的schema {schema}")
            others_gql_list.append(lists[i]["content"] )

unique_list_disease = list( set( disease_gql_list ) )
unique_list_nba = list( set( nba_gql_list ) )
unique_list_potter = list( set( potter_gql_list ) )
unique_list_other = list( set( others_gql_list ) )

all_gql = []
all_gql.extend( test_gql_list )
all_gql.extend( train_gql_list )    # 5116
unique_list = list(set(all_gql))    # 1300
unique_test_list = list( set(test_gql_list))    # 511
# save_data( unique_test_list, "all_test_unique_list.json")

from type_stat import nGQL_classify
# dict_keys(['INSERT', 'OTHER_WRITE', 'MATCH', 'FETCH PROP ON', 'LOOKUP ON', 'FIND', 'GET SUBGRAPH', 'GO', 'SHOW', 'MIXED', 'QUICK_RETURN', 'OTHER'])
class_dict = nGQL_classify( unique_list )

test_pair = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/nl_query_pair/test_pair.json", "json")
train_pair = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/nl_query_pair/train_pair.json", "json")

In [2]:
class_dict_nba = nGQL_classify( unique_list_nba )
class_dict_nba

{'INSERT': ['INSERT VERTEX t2 (name, age) VALUES "11":("n2", 13)',
  'INSERT VERTEX school(name, found_time) VALUES "DUT":("DUT", timestamp("1988-03-01T08:00:00"))',
  'INSERT VERTEX t1(a, b) VALUES "player100":("Hello", 100)',
  'INSERT EDGE follow(degree)\nVALUES "player100"->"player101":(95)',
  'INSERT EDGE e1(p1) VALUES "1"->"2"@0:(10)',
  'INSERT EDGE IF NOT EXISTS like(likeness) VALUES "Tom"->"Conan":(10)',
  'INSERT VERTEX person(p1) VALUES "1":(1)',
  'INSERT VERTEX player(name, age) VALUES "player101":("Tony Parker", 36)',
  'INSERT VERTEX school(name, found_time) VALUES "dut":("dut", now())',
  'INSERT VERTEX person(p1) VALUES "2":(2)',
  'INSERT EDGE like VALUES "Tom"->"Conan":(200)',
  'INSERT VERTEX player(name, age) VALUES "player100":("Tim Duncan", 42)',
  'INSERT VERTEX team(name) VALUES "team203":("Trail Blazers"), "team204":("Spurs")',
  'insert edge serve(start_year,end_year)\nvalues "player100"->"team204"@1:(1998, 2017)',
  'INSERT VERTEX t5(p1, p2, p3) VALUES "001

In [3]:
class_dict_nba.keys()

dict_keys(['INSERT', 'OTHER_WRITE', 'MATCH', 'FETCH PROP ON', 'LOOKUP ON', 'FIND', 'GET SUBGRAPH', 'GO', 'SHOW', 'MIXED', 'QUICK_RETURN', 'OTHER'])

In [11]:
def get_nl_from_gql( gql ):
    for i in range(len(test_pair)):
        if gql == test_pair[i]["查询"]:
            return test_pair[i]["用户"]
    for i in range(len(train_pair)):
        if gql == train_pair[i]["查询"]:
            return train_pair[i]["用户"]
    return "没找到"

## 抽取意图

In [12]:
os.chdir( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions")
from query_parser_Cypher import parse_match_query
from type_stat import nGQL_classify, get_intention
from query_parser_nGQL import (
    split_ngql, parser_yield, parser_where, parse_lookup_query, 
    parse_fetch_query, parse_find_query, parse_subgraph_query, parse_write_query,
    parse_go_query, parse_mixed_to_standard
)

from query_parser_standard import (
    get_standard_list, parse_fetch_to_standard, parse_look_to_standard, parse_find_to_standard,
    parse_subgraph_to_standard, parse_go_to_standard, parse_wirte_other_to_standard
)

def count_parsed_number( parsed_dict ):
    count = 0
    for key in parsed_dict:
        count += len(parsed_dict[key])
    return count

# dict_keys(['INSERT', 'OTHER_WRITE', 'MATCH', 'FETCH PROP ON', 'LOOKUP ON',
#  'FIND', 'GET SUBGRAPH', 'GO', 'SHOW', 'MIXED', 'QUICK_RETURN', 'OTHER'])

from query_parser_standard import parse_intention
parsed_dict, query_dict = parse_intention(class_dict_nba )
count_parsed_number( parsed_dict )

597

## 形成意图字典

In [13]:
path = "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract"

all_dict = {}
for part_ in ["disease", "nba", "potter", "other"]:
    sub_dir = os.path.join( path, part_)
    files = os.listdir(sub_dir)
    for file in files:
        if "intention_list.json" in file:
            try:
                intention_list = load_data( os.path.join(sub_dir,  file), "json")
            except:
                raise Exception( f"加载失败{os.path.join(sub_dir,  file)}")
            query_list = load_data( os.path.join(sub_dir,  file).replace("intention_list", "query_list"), "json")
            if len( intention_list) != len(query_list):
                raise Exception( f"长度不一致{os.path.join(sub_dir,  file)} {len( intention_list), len(query_list)}")
            for i in range(len(query_list)):
                query = query_list[i]
                '''if query in all_dict:
                    raise Exception( f"已存在键{os.path.join(sub_dir,  file)} {query}")'''
                all_dict[query] = intention_list[i]

In [15]:
for lists, part in zip( [tests, trains], ["test", "train"]):
    ret_list = []
    save_path = f"/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/N3/intentions/intention_extract/intentions_label_{part}.json"
    for i in range(len(lists)):
        query = lists[i]['content']
        if query not in all_dict:
            raise Exception( f"不存在键{part} {query}")
        ret_list.append( all_dict[query])
    print(len(ret_list))
    save_data( ret_list, save_path)

1254
3862
